In [6]:
import cv2, time, os, tensorflow as tf

import numpy as np

from tensorflow.python.keras.utils.data_utils import get_file

import shutil
from PIL import Image
import os


np.random.seed(123)


class Detector:
    def _init_(self):
        pass
    def readClasses(self, classesFilePath):
        with open(classesFilePath, 'r') as f:
            self.classesList = f.read().splitlines()
        
        #colors list
        self.colorList = np.random.uniform(low = 0, high = 255, size = (len(self.classesList), 3))
    
    def downloadModel(self, modelURL):
        
        fileName = os.path.basename(modelURL)
        self.modelName = fileName[:fileName.index('.')]
    
        
        self.cacheDir = "/projects/rkraig/phoenix_venv/Object_detection/pretrained_models"
        
        os.makedirs(self.cacheDir)
        
        get_file(fname = fileName, 
                origin = modelURL, cache_dir = self.cacheDir, cache_subdir = "checkpoints", extract = True)
        
    def loadModel(self):
        print("Loading Model " + self.modelName)
        tf.keras.backend.clear_session()
        self.model = tf.saved_model.load(os.path.join(self.cacheDir, "checkpoints", self.modelName, "saved_model"))
        
        print("Model " + self.modelName + "loaded successfully...")
        
    def createBoundingBox(self, image, threshold = 0.5):
        inputTensor = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
        inputTesnor = tf.convert_to_tensor(inputTensor, dtype = tf.uint8)
        inputTensor = inputTensor[tf.newaxis, ...]
        
        detections = self.model(inputTensor)
        
        bboxs = detections['detection_boxes'][0].numpy()
        classIndexes = detections['detection_classes'][0].numpy().astype(np.int32)
        classScores = detections['detection_scores'][0].numpy()
        
        
        imH, imW, imC = image.shape
        
        bboxIdx = tf.image.non_max_suppression(bboxs, classScores, max_output_size= 50,
                                              iou_threshold = threshold, score_threshold = threshold)
        print(bboxIdx)
        
        if len(bboxIdx) != 0:
            for i in bboxIdx:
                bbox = tuple(bboxs[i].tolist())
                classConfidence = round(100*classScores[i])
                classIndex = classIndexes[i]
                
                classLabelText = self.classesList[classIndex].upper()
                classColor = self.colorList[classIndex]
                
                displayText = '{}: {}%'.format(classLabelText, classConfidence)
                
                ymin, xmin, ymax, xmax = bbox
                
                xmin, xmax, ymin, ymax = (xmin * imW, xmax * imW, ymin * imH, ymax * imH)
                xmin, xmax, ymin, ymax = int(xmin), int(xmax), int(ymin), int(ymax)
                
                cv2.rectangle(image, (xmin, ymin), (xmax,ymax), color = classColor, thickness = 1)
                cv2.putText(image, displayText, (xmin, ymin - 10), cv2.FONT_HERSHEY_PLAIN, 1, classColor, 2)
                
                ###############################################
                
                lineWidth = min(int((xmax-xmin)*0.2), int ((ymax - ymin)*0.2))
                
                cv2.line(image, (xmin, ymin), (xmin + lineWidth, ymin), classColor, thickness = 5)
                cv2.line(image, (xmin, ymin), (xmin, ymin + lineWidth), classColor, thickness = 5)
                
                cv2.line(image, (xmax, ymin), (xmax - lineWidth, ymin), classColor, thickness = 5)
                cv2.line(image, (xmax, ymin), (xmax, ymin + lineWidth), classColor, thickness = 5)
        
                ###############################################
                
                cv2.line(image, (xmin, ymax), (xmin + lineWidth, ymax), classColor, thickness = 5)
                cv2.line(image, (xmin, ymax), (xmin, ymax - lineWidth), classColor, thickness = 5)
                
                cv2.line(image, (xmax, ymax), (xmax - lineWidth, ymax), classColor, thickness = 5)
                cv2.line(image, (xmax, ymax), (xmax, ymax - lineWidth), classColor, thickness = 5)
                
        return image
    
        
    def predictImage(self, imagePath, threshold = 0.5):
        image = cv2.imread(imagePath)
        
        bboxImage = self.createBoundingBox(image, threshold)
        
        #cv2.imwrite(self.modelName + ".jpg", bboxImage)
        path="/projects/rkraig/phoenix_venv/Object_detection/Output"
        cv2.imwrite(os.path.join(path ,self.modelName + ".jpg"), bboxImage)
        #img = Image.open(bboxImage)
        #img.show()
        #cv2.imshow("Result", bboxImage)
        #cv2.waitkey(0)
        #cv2.destroyAllWindows()


In [7]:
modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz"

classFile = "/projects/rkraig/phoenix_venv/Object_detection/coco_names.txt"
imagePath = "/projects/rkraig/phoenix_venv/Object_detection/test/000000019320[1]_L.jpg"
threshold = 0.5

detector = Detector()
detector.readClasses(classFile)
detector.downloadModel(modelURL)
detector.loadModel()
detector.predictImage(imagePath, threshold)

51847168/51839363 [==============================] - 0s 0us/step
Loading Model efficientdet_d1_coco17_tpu-32


Model efficientdet_d1_coco17_tpu-32loaded successfully...
tf.Tensor([0 1], shape=(2,), dtype=int32)
